In [1]:
# Leetcode의 Schema를 mysql의 형태로 바꿔주는 코드

import re

def convert_to_mysql_format(sql_script):
    # 정수 값에 따옴표 제거
    sql_script = re.sub(r"values \('(\d+)'", r"values (\1", sql_script, flags=re.IGNORECASE)

    # 데이터 타입 변환 (예시: VARCHAR2 -> VARCHAR)
    sql_script = re.sub(r"VARCHAR2\((\d+)\)", r"VARCHAR(\1)", sql_script, flags=re.IGNORECASE)

    # 중복된 데이터 삽입 방지 (이 부분은 복잡할 수 있으며, 상황에 따라 다를 수 있음)
    
    # 주석 처리 변환 (예시: -- -> #)
    sql_script = sql_script.replace('--', '#')

    # 바이너리 데이터 처리 (예시: RAW -> BINARY)
    sql_script = re.sub(r"RAW\((\d+)\)", r"BINARY(\1)", sql_script, flags=re.IGNORECASE)

    # 기타 구문 변환 (특정 상황에 따라 필요한 경우 추가)

    return sql_script

##### 1581. Customer Who Visited but Did Not Make Any Transactions


```sql
SELECT v.customer_id, COUNT(v.visit_id) AS count_no_trans
FROM Visits v
LEFT JOIN Transactions t ON v.visit_id = t.visit_id
WHERE t.transaction_id IS NULL
GROUP BY v.customer_id;
``````

##### 197. Rising Temperature

- 같은 테이블 내에서 전 날보다 기온이 낮은 열들을 추출하는 문제

- Weather a의 복제본 Weather b를 만든다.
- 기존 테이블에 DATE_ADD(b.recordDate, INTERVAL 1 DAY)로 하루씩 밀린 날짜 column을 더해준다.
- 위에서 만든 column을 기준으로 ON하여 두 테이블을 JOIN해준다.
- WHERE로 a.temperature > b.temperature 설정

```sql
SELECT a.id
FROM Weather a
     JOIN Weather b ON a.recordDate = DATE_ADD(b.recordDate, INTERVAL 1 DAY)
WHERE a.temperature > b.temperature;
``````

##### 1661. Average Time of Process per Machine


```sql
SELECT 
    start.machine_id, 
    ROUND(AVG(end.timestamp - start.timestamp), 3) AS processing_time
FROM 
    Activity AS start
JOIN 
    Activity AS end ON start.machine_id = end.machine_id 
                    AND start.process_id = end.process_id 
                    AND start.activity_type = 'start' 
                    AND end.activity_type = 'end'
GROUP BY 
    start.machine_id;
``````

아래는 위처럼 간단하게 할 수 있는 테이블을 하나하나 생성하며 헤멘 버전... 어지간한건 join으로 한번에 해결 가능

```sql
CREATE TABLE s AS
SELECT machine_id, process_id, timestamp
FROM Activity
WHERE activity_type = 'start';

CREATE TABLE e AS
SELECT machine_id, process_id, timestamp
FROM Activity
WHERE activity_type = 'end';

CREATE TABLE ProcessDuration AS
SELECT s.machine_id, 
       s.process_id, 
       e.timestamp - s.timestamp AS duration
FROM s
JOIN e ON s.machine_id = e.machine_id AND s.process_id = e.process_id;

SELECT machine_id, ROUND(AVG(duration),3) processing_time
FROM ProcessDuration
GROUP BY machine_id;
```

위처럼 여러개의 쿼리로 이루어져 있으면 Leetcode에서 오류가난다.\
\
with을 이용하여 하나의 쿼리로 합쳐주면 해결된다.

```sql
WITH s AS (
    SELECT machine_id, process_id, timestamp
    FROM Activity
    WHERE activity_type = 'start'
),
e AS (
    SELECT machine_id, process_id, timestamp
    FROM Activity
    WHERE activity_type = 'end'
),
ProcessDuration AS (
    SELECT s.machine_id, 
           s.process_id, 
           e.timestamp - s.timestamp AS duration
    FROM s
    JOIN e ON s.machine_id = e.machine_id AND s.process_id = e.process_id
)
SELECT machine_id, ROUND(AVG(duration), 3) AS processing_time
FROM ProcessDuration
GROUP BY machine_id;
``````

##### 577. Employee Bonus

```sql
SELECT e.name, b.bonus
FROM Employee e
LEFT JOIN Bonus b ON e.empId = b.empId
GROUP BY e.name, b.bonus
HAVING b.bonus < 1000 or b.bonus IS NULL;
```

위 코드가 testcase는 통과하였으나 추가적인 검증에는 fail이 떴다\
\
문제점
- empId로 이미 고유하게 식별되므로, GROUP BY를 사용할 필요가 없다.
- 보너스 정보가 전혀 없는 직원들을 제대로 처리하지 못했을 가능성이 있음.
- --> WHERE 사용


```sql
SELECT e.name, b.bonus
FROM Employee e
LEFT JOIN Bonus b ON e.empId = b.empId
WHERE b.bonus < 1000 OR b.bonus IS NULL;
```

##### 1280. Students and Examinations

- CROSS JOIN을 통해
```sql
CROSS JOIN Subjects sub
```
로 Students 테이블과 Subjects 테이블을 합쳐주며 row 값을 최대치로 늘려놓고 채워넣는 방향으로 한다.

- LEFT JOIN을 해야 Studant_id가 누락 없이 전부 나온다.

```sql
SELECT st.student_id, st.student_name, sub.subject_name, COUNT(ex.student_id) AS attended_exams
FROM Students st
CROSS JOIN Subjects sub
LEFT JOIN Examinations ex ON st.student_id = ex.student_id AND sub.subject_name = ex.subject_name
GROUP BY st.student_id, st.student_name, sub.subject_name
ORDER BY st.student_id, sub.subject_name;
```
* 전체를 JOIN한 테이블에서 Examinations의 student_id를 기준으로 COUNT해준다.


정보: MYSQL에서 GROUP BY는 ONLY_FULL_GROUP_BY모드를 지원하며, SELECT의 집계함수를 제외한 모든 항목들이 GROUP BY에 포함되어야 한다.

##### 570. Managers with at Least 5 Direct Reports

1. managerId를 기준으로 직원을 그룹화, COUNT로 5 이상만 남김 (서브쿼리로 생성)
2. 위에서 만든 서브쿼리 안에 id가 들어가는지로 파악
3. 위 조건을 만족하는 name만 추출

```sql
SELECT name
FROM Employee
WHERE id IN (
	SELECT e.managerId
	FROM Employee e
	WHERE e.managerId IS NOT NULL
	GROUP BY e.managerId
	HAVING COUNT(*) >=5
	);
``````